# 05. 감성 분석 (Sentiment Analysis)

p491(510) ~

In [1]:
import warnings
warnings.filterwarnings('ignore')

<br>

## 5.1 감성 분석 소개

- 문서의 주관적인 감성/의견/감정/기분 등을 파악하기 위한 방법
- 소셜 미디어, 여론조사, 온라인 리뷰, 피드백 등 다양한 분야에서 활용되고 있음

- 감성 분석은 문서 내 텍스트가 나타내는 여러 가지 주관적인 단어와 문맥을 기반으로 감성(Sentiment) 수치를 계산하는 방법을 이용
- 이러한 **감성 지수**는 긍정 감성 지수와 부정 감성 지수로 구성
- 이들 지수를 합산해 긍정 감성 또는 부정 감성을 결정

<br>

### 5.1.1 감성 분석의 학습 방식 구분

감성 분석은 머신러닝 관점에서 **지도학습**과 **비지도학습** 방식으로 나눌 수 있음

**지도학습**

- 학습 데이터와 타깃 레이블 값을 기반으로 감성 분석을 수행
- 이를 기반으로 다른 데이터의 감성 분석을 예측하는 방법
- 일반적인 텍스트 기반의 분류와 거의 동일

**비지도학습**

- 'Lexicon' 이라는 일종의 감성 어휘 사전을 이용
- Lexicon은 감성 분석을 위한 용어와 문맥에 대한 다양한 정보를 가지고 있음
- 이를 이용해 문서의 긍정적, 부정적 감성 여부를 판단

<br>

## 5.2 지도학습 기반 감성 분석 실습 - IMDB 영화평

- 지도학습 기반으로 감성 분석 수행
- 유명한 IMDB의 영화 사이트의 영화평을 이용
  - 지도학습 기반 감성 분석은 텍스트 기반의 이진 분류라고 표현할 수 있음
- 영화평의 텍스트를 분석해 감성 분석 결과가 긍정 또는 부정인지를 예측하는 모델 생성
- 캐글의 ["Bag of Words Meets Bags of Popcorn"](https://www.kaggle.com/c/word2vec-nlp-tutorial/data) Competition에서 데이터 다운로드

<br>

### 5.2.1 데이터셋 로드

In [2]:
import pandas as pd

review_df = pd.read_csv('data/IMDB/labeledTrainData.tsv',
                        header=0,
                        sep='\t',
                        quoting=3)
review_df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


- `id`
  - 각 데이터의 id  
  
  
- `sentiment`
  - 영화평(review)의 Sentiment 결과 값 (Target Label)
  - `1` : 긍정적 평가
  - `0` : 부정적 평가  
  
  
- `review`
  - 영화평의 텍스트

<br>

### 5.2.2 텍스트 데이터 전처리

텍스트가 어떻게 구성돼 있는 지 `review` 컬럼의 텍스트 값 하나 확인

In [3]:
print(review_df['review'][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [4]:
review_df['review'].str.contains("<br />").value_counts()

True     14665
False    10335
Name: review, dtype: int64

<br>

#### 5.2.2.1 제거 대상 1 : HTML 태그

- HTML 형식에서 추출해 `<br/>` 태그가 존재한다.  
$\Rightarrow$ `<br/>` 문자열은 피처로 만들 필요가 없으므로 삭제  
$\Rightarrow$ `DataFrame`/`Series` 객체에 `str.replace()`를 사용해 `<br/>` 태그를 공백으로 치환

In [5]:
# <br> html 태그는 replae 함수로 공백으로 변환
review_df['review'] = review_df['review'].str.replace('<br />', ' ')

<br>

#### 5.2.2.2 제거 대상 2 : 영어가 아닌 숫자/특수문자

- 영어가 아닌 숫자/특수문자 또한 Sentiment를 위한 피처로는 별 의미가 없어 보이므로 공란으로 변경
- 파이썬 `re` 모듈을 통해 정규 표현식을 이용하여 치환
  - 정규 표현식 `[^a-zA-Z]` : 영어 대/소문자가 아닌 모든 문자
  - `re.sub("[^a-zA-Z]", " ", x)` : 영어 대/소문자가 아닌 모든 문자를 찾아서 공란으로 변경

In [6]:
# 파이썬의 정규 표현식 모듈인 re를 이용해 영어 문자열이 아닌 문자는 모두 공백으로 변환
import re

review_df['review'] = review_df['review'].apply(lambda x: re.sub("[^a-zA-Z]", " ", x))

<br>

### 5.2.3 데이터 세트 분리

- 결정 값 클래스인 `sentiment` 컬럼을 별도로 추출해 **결정 값 데이터 세트** 생성
- 원본 데이터 세트에서 `id`와 `sentiment` 컬럼을 삭제해 **피처 데이터 세트** 새엇ㅇ
- `train_test_split()`을 이용해 학습용과 테스트용 데이터 세트로 분리

In [7]:
from sklearn.model_selection import train_test_split

class_df = review_df['sentiment']
feature_df = review_df.drop(['id', 'sentiment'], axis=1, inplace=False)

X_train, X_test, y_train, y_test = train_test_split(feature_df, class_df,
                                                    test_size=0.3, random_state=156)

print(X_train.shape, X_test.shape)

(17500, 1) (7500, 1)


- 학습용 데이터는 17,500개의 리뷰로 구성
- 테스트용 데이터는 7,500개의 리뷰로 구성

<br>

### 5.2.4 피처 벡터화 및 분류 알고리즘 적용

- 감상평(Review) 텍스트를 피처 벡터화 실시
- 그런 다음 ML 분류 알고리즘을 적용해 예측 성능을 측정
- `Pipeline` 객체를 이용해 이 두 가지를 한꺼번에 수행

<br>

**피처 벡터화**

- Count 벡터화
- TF-IDF 벡터화

**분류 알고리즘(Classifier)**

- `LogisticRegression`

**예측 성능 평가**

- 테스트 세트의 정확도
- ROC-AUC

<br>

#### 5.2.4.1 분류 모델 생성/예측/평가 (`CountVectorizer`를 이용한 피처 벡터화)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# 스톱 워드는 English, filtering, ngram은 (1,2)로 설정해 CountVectorization 수행
# LogisticRegression의 C는 10으로 설정
pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lr_clf', LogisticRegression(C=10))
])

# Pipeline 객체를 이용해 fit(), predict()로 학습/예측 수행.
# predict_proba()는 roc_auc 때문에 수행
pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print("예측 정확도 : {0:.4f}, ROC-AUC : {1:.4f}".format(accuracy_score(y_test, pred),
                                                        roc_auc_score(y_test, pred_probs)))

예측 정확도 : 0.8859, ROC-AUC : 0.9503


<br>

#### 5.2.4.2 분류 모델 생성/예측/평가 (`TfidfVectorizer`를 이용한 피처 벡터화)

In [10]:
# 스톱 워드는 english, filtering, ngram은 (1,2)로 설정해 TF-IDF 벡터화 수행
# LogisticRegression의 C는 10으로 설정
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lr_clf', LogisticRegression(C=10))
])

pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print("예측 정확도 : {0:.4f}, ROC-AUC : {1:.4f}".format(accuracy_score(y_test, pred),
                                                        roc_auc_score(y_test, pred_probs)))

예측 정확도 : 0.8936, ROC-AUC : 0.9598


<br>

#### 5.2.4.3 피처 벡터화 방법별 평가 지표 비교

| 피처 벡터화 | 정확도 | ROC-AUC |
| ----------- | ------ | ------- |
| Count       | 0.8860 | 0.9503  |
| TF-IDF      | 0.8936 | 0.9598  |

<br>

## 5.3 비지도학습 기반 감성 분석 소개

- 비지도 감성 분석은 Lexicon을 기반으로 함
- 많은 감성 분석용 데이터는 결정된 레이블 값을 가지고 있지 않음
- 이러한 경우에 Lexicon은 유용하게 사용될 수 있다.  
(한글을 지원하는 Lexicon은 없음)

<br>

### 5.3.1 Lexicon

- 일반적으로 어휘집을 의미
- 여기서는 주로 감성만을 분석하기 위해 지원하는 감성 어휘 사전 (감성 사전)
- 감성 사전은 긍정(Positive) 감성 또는 부정(Negative) 감성의 정도를 의미하는 수치를 가지고 있음  
$\Rightarrow$ 이를 **감성 지수(Polarity score)**라고 한다.
- 감성 지수는 단어의 위치나 주변 단어, 문맥, POS(Part of Speech) 등을 참고해 결정됨
- 감성 사전을 구현한 대표적인 패키지 : `NLTK`
  - `NLTK`는 많은 서브 모듈을 가지고 있음
  - 그 중에 감성 사전인 `Lexicon` 모듈도 포함돼 있음

<br>

### 5.3.2 `WordNet`

- `NLP` 패키지에서 제공하는 방대한 영어 어휘 사전
- `WordNet`은 단순한 어휘 사전이 아닌 **시멘틱 분석을 제공**하는 어휘 사전

<br>

**(참고) 시멘틱(semantic)**

- 텍스트 분석(Text Analytics)에서 자주 사용되는 용어
- "문맥상 의미"
  - "말" 이란 것은 문맥에 따라, 화자의 몸짓이나 어조에 따라 다르게 해석될 수 있음
  - 동일한 단어나 문장이라도 다른 환경과 문맥에서는 다르게 표현되거나 이해될 수 있음
- 언어학에서 이러한 시멘틱을 표현하기 위해 여러 가지 규칙을 정해왔음
- `NLP` 패키지는 시맨틱을 프로그램적으로 인터페이스할 수 있는 다양한 방법 제공

<br>

- `WordNet`은 다양한 상황에서 같은 어휘라도 다르게 사용되는 어휘의 시맨틱 정보를 제공
- 이를 위해 각각의 품사(명사, 동사, 형용사, 부사 등)로 구성된 개별 단어를 **Synset(Sets of congnitive synonyms)**라는 개념을 이용해 표현
- Synset : 단순한 하나의 단어가 아니라 그 단어가 가지는 문맥, 시맨틱 정보를 제공하는 `WordNet`의 핵심 개념

<br>

### 5.3.3 대표적인 감성 사전

- `NLTK`는 예측 성능이 그리 좋지 못한 단점이 있음
- 실제 업무의 적용에는 다른 감성 사전들이 적용됨

<br>

#### 5.3.3.1 `SentiWordNet`

- `NLTK` 패키지의 `WordNet`과 유사하게 감성 단어 전용의 WordNet을 구현한 것  
(`WordNet`의 Synset 개념을 감성 분석에 적용)
- `WordNet`의 Synset 별로 3가지 감성 점수(sentiment score)를 할당
  - 긍정 감성 지수
  - 부정 감성 지수
  - 객관성 지수 : 긍정/부정 감성 지수와 완전히 반대되는 개념(단어가 감성과 관계없이 얼마나 갠관적인지)
- 문장별로 단어들의 긍정 감성 지수와 부정 감성 지수를 합산하여 최종 감성 지수를 계산
- 이를 기반으로 감성이 긍정인 지 부정인 지 결정

<br>

#### 5.3.3.2 `VADER`

- 주로 소셜 미디어의 텍스트에 대한 감성 분석을 제공하기 위한 패키지
- 뛰어난 감성 분석 결과를 제공
- 비교적 빠른 수행 시간을 보장 $\Rightarrow$ 대용량 텍스트 데이터에 잘 사용됨

<br>

#### 5.3.3.3 `Pattern`

- 예측 성능 측면에서 가장 주목받는 패키지
- 현재 기준, 파이썬 3.X 버전에서 호환되지 않음 (파이썬 2.X 버전에서만 동작)

<br>

## 5.4 `SentiWordNet`을 이용한 감성 분석

### 5.4.1 `WordNet Synset`과 `SentiWordNet SentiSynset` 클래스의 이해

- `WordNet`을 이용하기 위해서는 `NLTK`를 셋업한 후에 `WordNet` 서브 패키지와 데이터 세트를 내려받아야 한다.

In [9]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to C:\Users\krx-
[nltk_data]    |     mktinfo\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to C:\Users\krx-
[nltk_data]    |     mktinfo\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to C:\Users\krx-
[nltk_data]    |     mktinfo\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to C:\Users\krx-
[nltk_data]    |     mktinfo\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\brown.zip.
[nltk_data]    | Downloading package brown_tei to C:\Users\krx-
[nltk_data]    |     mktinfo\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to C:\Users\krx-
[nltk_data]    |     m

True

<br>

`NLTK`의 모든 데이터 세트를 내려받은 뒤에 `WordNet` 모듈을 임포트해서 "present" 단어에 대한 `Synset` 추출

- `WordNet`의 `synsets()`는 파라미터로 지정된 단어에 대해 `WordNet`에 등재된 모든 `Synset` 객체를 반환

In [10]:
from nltk.corpus import wordnet as wn

term = 'present'

# 'present' 라는 단어로 wordnet의 synsets 생성
synsets = wn.synsets(term)
print('synsets() 반환 type : ', type(synsets))
print('synsets() 반환 값 개수 : ', len(synsets))
print('synsets() 반환 값 : ', synsets)

synsets() 반환 type :  <class 'list'>
synsets() 반환 값 개수 :  18
synsets() 반환 값 :  [Synset('present.n.01'), Synset('present.n.02'), Synset('present.n.03'), Synset('show.v.01'), Synset('present.v.02'), Synset('stage.v.01'), Synset('present.v.04'), Synset('present.v.05'), Synset('award.v.01'), Synset('give.v.08'), Synset('deliver.v.01'), Synset('introduce.v.01'), Synset('portray.v.04'), Synset('confront.v.03'), Synset('present.v.12'), Synset('salute.v.06'), Synset('present.a.01'), Synset('present.a.02')]


<br>

#### 5.4.1.1 `synsets()` 호출 시 반환되는 것

- 여러 개의 Synset 객체를 가지는 리스트
- 총 18개의 서로 다른 semantic을 가지는 synset 객체가 반환됨

<br>

#### 5.4.1.2 반환 값의 의미 (`present.n.01`)

- `Synset` 객체의 파라미터 `present.n.01`은 POS 태그를 나타냄
- `present` : 의미
- `n` : 명사 품사
- `01` : present가 명사로서 가지는 여러 가지의 의미를 구분하는 인덱스

<br>

#### 5.4.1.3 `synset` 객체가 가지는 여러 가지 속성

- `Synset`은 POS(Part of Speech, 품사), 정의(Definition), 부명제(Lemma) 등으로 시맨틱적인 요소를 표현할 수 있다.

In [11]:
for synset in synsets:
    print("##### Synset name : ", synset.name, "#####")
    print("POS : ", synset.lexname())
    print("Definition : ", synset.definition())
    print("Lemmas : ", synset.lemma_names())
    print("")

##### Synset name :  <bound method Synset.name of Synset('present.n.01')> #####
POS :  noun.time
Definition :  the period of time that is happening now; any continuous stretch of time including the moment of speech
Lemmas :  ['present', 'nowadays']

##### Synset name :  <bound method Synset.name of Synset('present.n.02')> #####
POS :  noun.possession
Definition :  something presented as a gift
Lemmas :  ['present']

##### Synset name :  <bound method Synset.name of Synset('present.n.03')> #####
POS :  noun.communication
Definition :  a verb tense that expresses actions or states at the time of speaking
Lemmas :  ['present', 'present_tense']

##### Synset name :  <bound method Synset.name of Synset('show.v.01')> #####
POS :  verb.perception
Definition :  give an exhibition of to an interested audience
Lemmas :  ['show', 'demo', 'exhibit', 'present', 'demonstrate']

##### Synset name :  <bound method Synset.name of Synset('present.v.02')> #####
POS :  verb.communication
Definition :  bri

<br>

#### 5.4.1.4 개별 출력 결과 확인

**`Synset('present.n.01')`**

In [12]:
print("##### Synset name : ", synsets[0].name, "#####")
print("POS : ", synsets[0].lexname())
print("Definition : ", synsets[0].definition())
print("Lemmas : ", synsets[0].lemma_names())

##### Synset name :  <bound method Synset.name of Synset('present.n.01')> #####
POS :  noun.time
Definition :  the period of time that is happening now; any continuous stretch of time including the moment of speech
Lemmas :  ['present', 'nowadays']


- POS : `noun.time`
  - `noun` : 명사
- Definition : "시간적인 의미로 현재"

<br>

**`Synset('present.n.02')`**

In [13]:
print("##### Synset name : ", synsets[1].name, "#####")
print("POS : ", synsets[1].lexname())
print("Definition : ", synsets[1].definition())
print("Lemmas : ", synsets[1].lemma_names())

##### Synset name :  <bound method Synset.name of Synset('present.n.02')> #####
POS :  noun.possession
Definition :  something presented as a gift
Lemmas :  ['present']


- POS : `noun.possession`
  - `noun` : 명사
  - `possession` : 소유권
- Definition : "선물"

<br>

**`Synset('show.v.01')`**

In [14]:
print("##### Synset name : ", synsets[3].name, "#####")
print("POS : ", synsets[3].lexname())
print("Definition : ", synsets[3].definition())
print("Lemmas : ", synsets[3].lemma_names())

##### Synset name :  <bound method Synset.name of Synset('show.v.01')> #####
POS :  verb.perception
Definition :  give an exhibition of to an interested audience
Lemmas :  ['show', 'demo', 'exhibit', 'present', 'demonstrate']


- POS : `verb.perception`
  - `verb` : 동사
  - `perception` : 통찰력
- Definition : "관객에게 전시물 등을 보여주다."

<br>

이처럼 `synset`은 하나의 단어가 가질 수 있는 여러 가지 시맨틱 정보를 개별 클래스로 나타낸 것이다.

<br>

#### 5.4.1.5 어휘 간 관계 유사도

- `WordNet`은 어떤 어휘와 다른 어휘 간의 관계를 유사도로 나타낼 수 있다.
- `synset` 객체는 단어 간의 유사도를 나타내기 위해서 `path_similarity()` 메서드를 제공


<br>

`path_similarity()` 를 이용해 "tree", "lion", "tiger", "cat", "dog" 라는 단어의 상호 유사도 확인

In [15]:
# synset 객체를 단어별로 생성
tree = wn.synset("tree.n.01")
lion = wn.synset("lion.n.01")
tiger = wn.synset("tiger.n.02")
cat = wn.synset("cat.n.01")
dog = wn.synset("dog.n.01")

entities = [tree, lion, tiger, cat, dog]
similarities = []
entity_names = [entity.name().split(".")[0] for entity in entities]
entity_names

['tree', 'lion', 'tiger', 'cat', 'dog']

In [16]:
# 단어별 synset을 반복하면서 다른 단어의 synset과 유사도를 측정
for entity in entities:
    similarity = [round(entity.path_similarity(compared_entity), 2) for compared_entity in entities]
    similarities.append(similarity)
similarities

[[1.0, 0.07, 0.07, 0.08, 0.12],
 [0.07, 1.0, 0.33, 0.25, 0.17],
 [0.07, 0.33, 1.0, 0.25, 0.17],
 [0.08, 0.25, 0.25, 1.0, 0.2],
 [0.12, 0.17, 0.17, 0.2, 1.0]]

In [17]:
# 개별 단어별 synset과 다른 단어의 synset 과의 유사도를 DataFrame 형태로 저장
similarity_df = pd.DataFrame(similarities, columns=entity_names, index=entity_names)
similarity_df

,tree,lion,tiger,cat,dog
tree,1.00,0.07,0.07,0.08,0.12
lion,0.07,1.00,0.33,0.25,0.17
tiger,0.07,0.33,1.00,0.25,0.17
cat,0.08,0.25,0.25,1.00,0.20
dog,0.12,0.17,0.17,0.20,1.00


$\Rightarrow$ lion은 tree와의 유사도가 가장 적음 (0.07)  
$\Rightarrow$ lion은 tiger와 유사도가 가장 큼 (0.33)

<br>

#### 5.4.1.6 `Senti_Synset` 클래스

- `SentiWordNet`은 `WordNet`의 `Synset`과 유사한 `Senti_Synset` 클래스를 가지고 있음
- `SentiWordNet` 모듈의 `senti_synsets()`는 `WordNet` 모듈이라서 `synsets()`와 비슷하게 `Senti_Synset` 클래스를 리스트 형태로 반환한다.

In [18]:
import nltk
from nltk.corpus import sentiwordnet as swn

senti_synsets = list(swn.senti_synsets('slow'))

print("senti_synsets() 반환 type : ", type(senti_synsets))
print("senti_synsets() 반환 값 개수 : ", len(senti_synsets))
print("senti_synsets() 반환 값 : ", senti_synsets)

senti_synsets() 반환 type :  <class 'list'>
senti_synsets() 반환 값 개수 :  11
senti_synsets() 반환 값 :  [SentiSynset('decelerate.v.01'), SentiSynset('slow.v.02'), SentiSynset('slow.v.03'), SentiSynset('slow.a.01'), SentiSynset('slow.a.02'), SentiSynset('dense.s.04'), SentiSynset('slow.a.04'), SentiSynset('boring.s.01'), SentiSynset('dull.s.08'), SentiSynset('slowly.r.01'), SentiSynset('behind.r.03')]


<br>

#### 5.4.1.7 `SentiSynset` 객체의 감성 지수와 객관성 지수

- `SentiSynset` 객체는 단어의 감성을 나타내는 감성 지수와 객관성(감성과 반대)을 나타내는 객관성 지수를 가지고 있다.
- 감성 지수는 긍정 감성 지수와 부정 감성 지수로 나뉨
- 어떤 단어가 전혀 감성적이지 않다 $\rightarrow$ 객관성 지수 = 1, 감성 지수 = 0 (긍정, 부정 모두)

<br>

father(아버지)라는 단어와 fabulous(아주 멋진)라는 두 개 단어의 감성 지수와 객관성 지수를 나타냄

In [19]:
import nltk
from nltk.corpus import sentiwordnet as swn

father = swn.senti_synset('father.n.01')
print('father 긍정 감성 지수 : ', father.pos_score())
print('father 부정 감성 지수 : ', father.neg_score())
print('father 객관성 지수 : ', father.obj_score())

father 긍정 감성 지수 :  0.0
father 부정 감성 지수 :  0.0
father 객관성 지수 :  1.0


- father는 객관적인 단어로 객관성 지수가 1.0이고 긍정 감성/부정 감성 지수 모두 0이다.

<br>

In [20]:
fabulous = swn.senti_synset('fabulous.a.01')
print('fabulous 긍정 감성 지수 : ', fabulous.pos_score())
print('fabulous 부정 감성 지수 : ', fabulous.neg_score())
print('fabulous 객관성 지수 : ', fabulous.obj_score())

fabulous 긍정 감성 지수 :  0.875
fabulous 부정 감성 지수 :  0.125
fabulous 객관성 지수 :  0.0


- 반면에 fabulous는 감성 단어로서 긍정 감성 지수가 0.875, 부정 감성 지수가 0.125이다.

<br>

### 5.4.2 `SentiWordNet`을 이용한 영화 감상평 분석

IMDB 영화 감상평 감성 분석을 SentiWordNet Lexicon 기반으로 수행

<br>

#### 5.4.2.1 SentiWordNet을 이용해 감성 분석을 수행하는 순서

1. 문서(Document)를 문장(Sentence) 단위로 분해
2. 다시 문장(Sentence)을 단어(Word) 단위로 토큰화하고 품사 태깅
3. 품사 태깅된 단어 기반으로 `synset` 객체와 `senti_synset` 객체를 생성
4. `Senti_synset`에서 긍정 감성/부정 감성 지수를 구하고 이를 모두 합산해 특정 임계치 값 이상일 때 긍정 감성으로, 그렇지 않을 때는 부정 감성으로 결정

<br>

#### 5.4.2.2 어근 추출(Lemmatization) 및 품사 태깅 (POS Tagging)

- SentiWordNet을 이용하기 위해서 WordNet을 이용해 문서를 다시 단어로 토큰화한 뒤 **어근 추출(Lemmatization)**과 **품사 태깅(POS Tagging)**을 적용해야 한다.

품사 태깅을 수행하는 내부 함수(`penn_to_wn`) 생성

In [21]:
from nltk.corpus import wordnet as wn

# 간단한 NLTK PennTreebank Tag를 기반으로 WordNet 기반의 품사 Tag로 변환
def penn_to_wn(tag):
    if tag.startswith("J"):
        return wn.ADJ
    elif tag.startswith("N"):
        return wn.NOUN
    elif tag.startswith("R"):
        return wn.ADV
    elif tag.startswith("V"):
        return wn.VERB

<br>

이제 문서를 문장 $\rightarrow$ 단어 토큰 $\rightarrow$ 품사 태깅 후에 SentiSynset 클래스를 생성하고 Polarity Score를 합산하는 함수(`swn_polarity()`) 생성

- 각 단어의 긍정 감성 지수와 부정 감성 지수를 모두 합한 총 감성 지수가 0 이상일 경우 긍정 감성, 그렇지 않을 경우 부정 감성으로 예측

In [22]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

def swn_polarity(text):
    # 감성 지수 초기화
    sentiment = 0.0
    tokens_count = 0
    
    lemmatizer = WordNetLemmatizer()
    raw_sentences = sent_tokenize(text)
    
    # 분해된 문장별로 단어 토큰 -> 품사 태깅 후에 SentiSynset 생성 -> 감성 지수 합산
    for raw_sentence in raw_sentences:
        # NLTK 기반의 품사 태깅 문장 추출
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        
        for word, tag in tagged_sentence:
            # WordNet 기반 품사 태깅과 어근 추출
            wn_tag = penn_to_wn(tag)
            
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue
            
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            
            if not lemma:
                continue
                
            # 어근을 추출한 단어와 WordNet 기반 품사 태깅을 입력해 Synset 객체를 생성
            synsets = wn.synsets(lemma, pos=wn_tag)
            
            if not synsets:
                continue
                
            # sentiwordnet의 감성 단어 분석으로 감성 synset 추출
            # 모든 단어에 대해 금정 감성 지수는 + 로, 부정 감성 지수는 - 로 합산해 감성 지수 계산
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            sentiment += (swn_synset.pos_score() - swn_synset.neg_score())
            tokens_count += 1
            
    if not tokens_count:
        return 0
    
    # 총 score가 0 이상일 경우 긍정(Positive) 1, 그렇지 않을 경우 부정(Negative) 0 반환
    if sentiment >= 0:
        return 1
    
    return 0

<br>

#### 5.4.2.3 개별 문서의 긍정 및 부정 감성 예측

- IMDB 감상평의 개별 문서에 대해 `swn_polarity(text)` 함수 적용해 긍정 및 부정 감성을 예측
- 판다스의 `apply lambda` 구문을 이용해 `swn_polarity(text)`를 개별 감상평 텍스트에 적용
- `review_df` 를 그대로 이용
- `review_df`의 새로운 컬럼으로 `preds`를 추가해 이 컬럼에 `swn_polarity(text)`로 반환된 감성 평가를 담음
- 실제 감성 평가인 `sentiment` 컬럼과 `preds` 컬럼의 정확도, 정밀도, 재현율 값을 모두 측정

In [23]:
review_df.head()

,id,sentiment,review
0,"""5814_8""",1,With all this stuff going down at the moment ...
1,"""2381_9""",1,The Classic War of the Worlds by Timothy ...
2,"""7759_3""",0,The film starts with a manager Nicholas Bell...
3,"""3630_4""",0,It must be assumed that those who praised thi...
4,"""9495_8""",1,Superbly trashy and wondrously unpretentious ...


In [24]:
review_df['preds'] = review_df['review'].apply(lambda x: swn_polarity(x))
y_target = review_df['sentiment'].values
preds = review_df['preds'].values

In [25]:
y_target

array([1, 1, 0, ..., 0, 0, 1], dtype=int64)

In [26]:
preds

array([0, 1, 0, ..., 1, 0, 0], dtype=int64)

<br>

#### 5.4.2.4 성능 평가

- 3장 평가(Evaluation)에서 만들었던 분류 성능 평가 함수(`get_clf_eval()`) 함수를 이용

In [27]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve

def get_clf_eval(y_test, pred) :
    
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    
    # F1 스코어 추가
    f1 = f1_score(y_test, pred)
    
    auc = roc_auc_score(y_test, pred)
    
    print('오차 행렬')
    print(confusion)
    
    print('정확도 : {0:.4f}, 정밀도 : {1:.4f}, 재현율 : {2:.4f}, F1 : {3:.4f}, AUC : {4:.4f}'.format(accuracy, precision, recall, f1, auc))

- SentiWordNet의 감성 분석 예측 성능 확인

In [28]:
print("#### SentiWOrdNet 예측 성능 평가")
get_clf_eval(y_target, preds)

#### SentiWOrdNet 예측 성능 평가
오차 행렬
[[7668 4832]
 [3636 8864]]
정확도 : 0.6613, 정밀도 : 0.6472, 재현율 : 0.7091, F1 : 0.6767, AUC : 0.6613


<br>

정확도 지표를 포함한 전반적인 성능 평가 지표는 만족스러울 만한 수치는 아님

<br>

## 5.5 VADER를 이용한 감성 분석

### 5.5.1 VADER Lexicon

- 또 다른 Lexicon
- VADER는 소셜 미디어의 감성 분석 용도로 만들어진 룰 기반의 Lexicon
- VADER는 `SentimentIntensityAnalyzer` 클래스를 이용해 쉽게 감성 분석을 제공
- VADER 제공 패키지 2가지
  1. NLTK 패키지의 서브 모듈로 제공
    - `nltk.download('vader_lexicon')`
  2. 단독 패키지로 제공
    - `pip install vaderSentiment`
    - `from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer`

<br>

### 5.5.2 VADER의 간단한 사용법

- NLTK 서브 모듈로 `SentimentIntensityAnalyzer`를 임포트하고 간략하게 감상평 한 개만 감성 분석 수행

In [29]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

senti_analyzer = SentimentIntensityAnalyzer()
senti_scores = senti_analyzer.polarity_scores(review_df['review'][0])
print(senti_scores)

{'neg': 0.13, 'neu': 0.743, 'pos': 0.127, 'compound': -0.7943}


<br>

### 5.5.3 VADER 이용 감성 분석

VADER를 이용하면 매우 쉽게 감성 분석을 수행할 수 있다.

- `SentimentIntensityAnalyzer` 객체 생성
- 문서별로 `polarity_scores()` 메서드를 호출해 감성 점수를 구함
- 해당 문서의 감성 점수가 특정 임계값 이상이면 긍정, 그렇지 않으면 부정으로 판단

`SentimentIntensityAnalyzer` 객체의 `polarity_scores()` 메서드는 딕셔너리 형태의 감성 점수를 반환

- `neg` : 부정 감성 지수
- `neu` : 중립적인 감성 지수
- `pos` : 긍정 감성 지수
- `compound` : `neg`, `neu`, `pos` score를 적절히 조합해 -1에서 1 사이의 감성 지수를 표현한 값

`compound` score를 기반으로 부정 감성 또는 긍정 감성 여부를 결정

- 0.1 이상 : 긍정 감성
- 0.1 이하 : 부정 감성
- 상황에 따라 이 임계값(0.1)을 적절히 조정해 예측 성능을 조절

<br>

### 5.5.4 VADER 이용 IMDB 감성 분석 수행

`vader_polarity()` 함수 재생성

- 입력 파라미터로 영화 감상평 텍스트와 긍정/부정을 결정하는 임곗값(threshold)을 가짐
- `SentimentIntensityAnalyzer` 객체의 `polarity_scores()` 메서드를 호출해 감성 결과를 반환

In [30]:
def vader_polarity(review, threshold=0.1):
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)
    
    # compound 값에 기반해 threshold 입력값보다 크면 1, 그렇지 않으면 0 반환
    agg_score = scores['compound']
    final_sentiment = 1 if agg_score >= threshold else 0
    return final_sentiment

`review_df` DataFrame의 `apply lambda` 식을 통해 `vader_polarity()` 함수를 호출해 각 문서별로 감성 결과를 `vader_preds` 라는 `review_df`의 새로운 컬럼에 저장한 뒤, 저장된 감성 분석 결과를 기반으로 VADER의 예측 성능을 측정

In [31]:
# apply lambda 식을 이용해 레코드별로 vader_polarity()를 수행하고 결과를 vader_preds에 저장
review_df['vader_preds'] = review_df['review'].apply(lambda x: vader_polarity(x, 0.1))
y_target = review_df['sentiment'].values
vader_preds = review_df['vader_preds'].values

In [32]:
print("#### VADER 예측 성능 평가 ####")
get_clf_eval(y_target, vader_preds)

#### VADER 예측 성능 평가 ####
오차 행렬
[[ 6736  5764]
 [ 1867 10633]]
정확도 : 0.6948, 정밀도 : 0.6485, 재현율 : 0.8506, F1 : 0.7359, AUC : 0.6948


<br>

- 정확도가 SentiWordNet보다 향상(0.6613 $\rightarrow$ 0.6948)됨
- 재현율은 SentiWordNet보다 크게 향상(0.7091 $\rightarrow$ 0.8506)됨

<br>

### 5.5.5 SentiWordNet과 VADER의 감성 분석 평가 지표 비교

| 평가 지표    | 정확도 | 정밀도 | 재현율 | F1 스코어 | ROC-AUC |
| ------------ | ------ | ------ | ------ | --------- | ------- |
| SentiWordNet | 0.6613 | 0.6472 | 0.7091 | 0.6767    | 0.6613  |
| VADER        | 0.6948 | 0.6485 | 0.8506 | 0.7359    | 0.6948  |